In [2]:
import os
import math
import csv
import sys
import datetime
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
feature_num = 18
feature_list = ['AMB_TEMP', 'CH4', 'CO', 'NMHC', 'NO', 'NO2', 'NOx',
                'O3','PM10', 'PM2.5', 'RAINFALL', 'RH', 'SO2', 'THC',
                'WD_HR', 'WIND_DIREC', 'WIND_SPEED', 'WS_HR']
train_data_path = './train.csv'
test_data_path = './test.csv'
output_answer_path = './answer.csv'

class Data(object):
    
    def __init__(self, location, date, feature_dict):
        
        self.location = location
        self.date = date
        self.feature_dict = feature_dict
        self.str2num()
        self.PM2point5 = self.feature_dict['PM2.5']
        
    def str2num(self):   
        for feature in feature_list:
            if feature == 'RAINFALL':
                temp = self.feature_dict[feature]
                for i in range(len(temp)):
                    if self.feature_dict[feature][i] == 'NR':
                        self.feature_dict[feature][i] = 1.
                    else:
                        self.feature_dict[feature][i] = 1.01
                self.feature_dict[feature] = np.array(self.feature_dict[feature])
                continue
            temp = self.feature_dict[feature]
            for i in range(len(temp)):
                self.feature_dict[feature][i] = float(temp[i])
            self.feature_dict[feature] = np.array(self.feature_dict[feature])
            
    def feature2matrix(self):   
        x = np.ndarray(shape=(18, 24), dtype=float)
        cntr = 0
        for feature in feature_list:
            x[cntr] = self.feature_dict[feature]
            cntr += 1
            
        return np.asmatrix(x)
    
    def test_feature2matrix(self):      
        x = np.ndarray(shape=(18, 9), dtype=float)
        cntr = 0
        for feature in feature_list:
            x[cntr] = self.feature_dict[feature]
            cntr += 1
            
        return np.asmatrix(x)
    
    def get_matrix(self):
        self.x = self.feature2matrix()
        return self.x
    
    def test_get_matrix(self):
        self.x = self.test_feature2matrix()
        return self.x
    
    def get_PM2point5(self):
        return self.PM2point5

In [4]:
def PM2point5_dataset(train_data_path):
    train_data, feature_mean_dict, feature_std_dict = feature_scaling(read_train_data(train_data_path))
    x_matrix = None
    y_matrix = None
    cntr = 0
    for data in train_data:
        if cntr == 0:
            x_matrix = data.get_matrix()
            y_matrix = data.get_PM2point5()
            cntr += 1
            continue
        x_temp = data.get_matrix()
        y_temp = data.get_PM2point5()
        x_matrix = np.hstack((x_matrix, x_temp))
        y_matrix = np.hstack((y_matrix, y_temp))
        cntr += 1
        
    return x_matrix, y_matrix, feature_mean_dict, feature_std_dict

def read_train_data(train_data_path):
    train_data = []
    feature_value = []
    line_count = 0
    
    with open(train_data_path, encoding='big5') as train_data_csv:
        next(train_data_csv)
        reader = csv.reader(train_data_csv, delimiter=',')
        for line in reader:
            line_count += 1
            time = line[0].split('/')
            date = datetime.datetime(int(time[0]), int(time[1]), int(time[2]))
            location = line[1].strip()
            value = []
            for ele in line[3:]:
                value.append(ele)     
            feature_value.append(value)
            if line_count % 18 == 0:
                feature_dict = dict(zip(feature_list, feature_value))
                train_data.append(Data(location, date, feature_dict))
                feature_value = []
                
    return train_data

def read_test_data(test_data_path, feature_mean_dict, feature_std_dict):
    test_data = []
    feature_value = []
    line_count = 0
    with open(test_data_path, encoding='big5') as test_data_csv:
        reader = csv.reader(test_data_csv, delimiter=',')
        for line in reader:
            line_count += 1
            value = []
            for ele in line[2:]:
                value.append(ele)     
            feature_value.append(value)
            if line_count % 18 == 0:
                feature_dict = dict(zip(feature_list, feature_value))
                test_data.append(Data('', '0-0-0', feature_dict))
                feature_value = [] 
                
    for data in test_data:
        for feature in feature_list:
            data.feature_dict[feature] = (data.feature_dict[feature] - feature_mean_dict[feature]) / feature_std_dict[feature]
                
    return test_data

def feature_scaling(train_data):  
    feature_value = [np.array([])]*24
    feature_dict = dict(zip(feature_list, feature_value))
    feature_mean = []*24
    feature_mean_dict = dict(zip(feature_list, feature_value))
    feature_std = []*24
    feature_std_dict = dict(zip(feature_list, feature_value)) 
    
    for data in train_data:    
        for feature in feature_list:
            feature_dict[feature] = np.concatenate((feature_dict[feature], data.feature_dict[feature]))   
    for feature in feature_list:    
        if feature == 'RAINFALL':
            feature_mean_dict[feature] = feature_dict[feature].mean()
            feature_std_dict[feature] = feature_dict[feature].std()
            continue
        feature_mean_dict[feature] = feature_dict[feature].mean()
        feature_std_dict[feature] = feature_dict[feature].std()
    
    feature_dict = dict(zip(feature_list, feature_value))
    for data in train_data:
        for feature in feature_list:
            data.feature_dict[feature] = (data.feature_dict[feature] - feature_mean_dict[feature]) / feature_std_dict[feature]

    return train_data, feature_mean_dict, feature_std_dict

In [5]:
W_b_XX = [-1.3541099634037714, 0.89841430536428957, 0.56787414224134436, -0.41542174665008413, 0.26228492521417546, -0.24122028278094076, -0.28703401155176617, -0.36362169583757792, 0.77929042124188919, -0.52690834503000972, 0.080010798919435577, 0.1477384914712202, 0.068548788866817575, -0.11701452788052502, -0.056761374252940554, -0.13200876583484528, 0.11040880500944183, 0.50843155423194897, 0.070532694934129253, -0.052997149581204439, 0.065298741147982425, -0.14459544645056649, 0.15860888058241585, -0.042835541584580875, -0.13192947196007249, 0.090259064005995246, 0.40767124566499635, -0.52711465073669295, 0.25772435292131113, -0.097519760058397797, 0.22707278140202969, 0.19580225369612203, -0.37646012142482899, 0.067501323602518037, 0.11337086488182183, 0.11596810175126959, 0.69237439022963387, -0.073919446936930186, 0.093999076855262792, -0.11808976327909332, -0.13422281308689582, -0.17895099916444235, 0.036533649340592451, 0.31850120149776073, -0.36881590699629163, 2.0375386152885109, -0.63035458755039253, -0.58200651499578848, -0.44670792048406766, -0.34211364952385331, -0.015116102362833166, -0.54142702132405907, -0.25346744396079651, 1.1036205967632553, -2.0573041539655548, 0.10339208084526447, 0.59203829392664364, 0.27168095542516196, 0.3705268478524536, 0.069289657938486612, 0.068320727546627547, -0.19919271786419027, 0.76097934653395782, 0.14560054301979933, 0.1094217924148337, -0.17908360474157572, -0.29366984544433294, -0.056131312808464506, -0.50387765901545944, -0.59397995937100878, -0.1438130471495544, 1.9142780444136489, -0.00054750162015442493, 0.12586415890549901, -0.039835266776689825, 0.39222522928707804, -0.46205525036005635, -0.12748387690877785, 0.45807625541630181, -0.26631239344394109, 1.3178878537563654, -0.36720764304028525, 0.3868747764617495, 1.612283976040618, -2.3121491663370835, 0.43640688000790684, 5.4897106959452309, -7.4302859064827373, 0.98362874058745697, 14.802699791917565, -0.036748724153894831, -0.030840470012504992, 0.059618691982631235, -0.15181248286280197, 0.13296931164821993, 0.12631781874176343, -0.076295958034143435, -0.17502965485146216, -0.23347351610925846, -0.42377421800005777, 0.56238829893734343, 0.14125902401326182, -0.3299387859331615, -0.16443917693713131, 0.017140111094849555, -0.75625941195177704, 0.2018068973796445, 0.46811581129652552, -0.48990790365776377, 0.57180151402084889, -0.15727401714817002, -0.034821439820363072, -0.042103802901091635, 0.1222115153348074, -0.19196846373958165, 0.18038131201044977, 0.30846184667475435, 0.65330484789265708, -0.1787160027095098, -0.024025639093699872, -0.35193971033654159, 0.035826280295022078, 0.34903111267625309, 0.077394346966976946, -0.32649350568101415, -0.13810955769756827, -0.016687789043391115, 0.25736364137025181, -0.05477444403828409, 0.19508691454335925, 0.0057144830540503252, 0.14997441213136573, -0.22645151443881084, 0.11967540849829995, 0.017335469159650039, -0.17120925570204282, -0.083888130847256909, 0.063166014418494149, -0.20647472287775831, 0.076618672298242249, 0.032403709031574121, -0.014066924659769659, -0.24064548480166484, 0.07644082367060373, -0.16152747803909276, -0.1025648291379043, 0.1760339267529194, 0.06061699786756234, -0.14330159265196096, -0.12788942702191583, -0.047460671237857091, -0.035765067550271444, -0.067806816689362018, -0.01855347142808466, 0.19013779204087988, -0.11773300932904113, -0.21199027829934319, 0.025406294127542913, 0.3596745827708312, -0.051895814547656891, -0.21742633548286625, 0.10205574554686207, -55.031266747264333, 0.36147811625837878, 0.32312186434886586, -0.087130776586537298, 0.4885204289907909, -0.24414983233587664, 0.42242069139493976, -0.037919806550366968, 0.36633213205422488, 0.3558181868003808, -0.080568896426204351, 0.62458025964744512, 0.76556931558014674, 0.42971778420516238, 0.75567119988481213, 0.84501689858630724, 0.73005747869419413, 0.67461525781544041, 0.28633500155577651, 0.79051540970578094, 0.085833736596760316, 0.53703513876509701, 0.068527445359321065, 0.33478603740860224, 0.63900545323836544, 0.42967817333678743, 0.13007342769779451, 0.8361488107793329, 0.024479281514126278, 0.056394596024959956, 0.15740935061491695, 0.57520110438242944, 0.81194569384084192, 0.40147902814796216, 0.6080918224941233, 0.67617333672720648, 0.87948478823551213, 0.71989100128289729, 0.31722184154892402, 0.26238521400161235, 0.83991188406423722, 0.56428779452866096, 0.23209003824927221, 0.26693192943002136, 0.65099064567554299, 0.86965425864897772, 0.18993391622594211, 0.12970962293589469, 0.5043790595884573, 0.34850393969952115, 0.57931384335982727, 0.085001039656611474, 0.54912471048530109, 0.51952479086615, 0.43603830575896879, 0.61575908568123294, 0.56805838219357052, 0.15942845642276185, 0.89763035397303292, 0.59385660715346311, 0.37010988383660587, 0.10596727955278572, 0.63760916318438732, 0.17256543741115829, 0.65750785030260472, 0.42197399613746545, 0.92489857110917317, 0.50533861918979905, 0.58904729340768314, 0.28503292478095676, 0.45842656102254176, 0.12337035234379143, 0.53081423651980175, 0.7220972773854929, 0.32147240347655825, 0.10284970821161127, 0.31171716939106353, 0.33409576782565964, 0.81912163276139094, 0.80602000542977481, 0.19740011528905646, 0.57427511374774953, 0.068263502594375247, 0.15088943151234765, 0.78625926196998985, 0.86000394212380515, 0.90779608146449176, 0.55770621988158431, 0.36736743771198155, 0.64129074284835852, 0.63973536745274828, 0.45872663088918514, 0.61147900767851793, 0.43592868495493703, 0.51198088129111086, 0.23437271619048858, 0.45695146199254072, 0.67898514395609533, 0.90047022944350652, 0.53763074863052218, 0.74926182234147798, 0.7830036406655736, 0.24587898787237297, 0.79121784379550431, 0.51849855616381646, 0.81852893768762203, 0.72464052872918683, 0.2348448351835016, 0.46730755255692874, 0.66481053573944882, 0.055926702108311833, 0.14269232282265892, 0.85686296888378455, 0.86113260079651577, 0.26024590807359183, 0.88370144822747088, 0.43312940885497875, 0.51748705537850814, 0.58668532312489463, 0.25124090398131499, 0.082222218901453883, 0.23664510902478225, 0.68919337671222991, 0.3834268256950023, 0.34383765439074426, 0.17597366500070796, 0.34155522626769119, 0.61099158496256056, 0.40416368089453475, 0.12000212523374047, 0.686153714509807, 0.62969730863141649, 0.33691584684869186, 0.14828243564628218, 0.22438739027672996, 0.77762080748392437, 0.52026395373128453, 0.37279520399396265, 0.60235636416173288, 0.18652877116115466, 0.066395226270632005, 0.7807125158849143, 0.18493055164342448, 0.81159115196584297, 0.35238829951980771, 0.60839293449047194, 0.62797927370890338, 0.20113054100482416, 0.59791651185752004, 0.053960468831381607, 0.76799756354712179, 0.83236311627682036, 0.28246820397054884, 0.93782581136312138, 0.51940118358981702, 0.90736533244576578, 0.69215737531253119, 0.4144838808037215, 0.50290502796473568, 0.82861233354481723, 0.44415626270340136, 0.59389895786327762]
W_b_100 = [-3.9051669768146713, 0.95407035524961425, 0.57417702662460279, 0.42419250182146967, 0.11506686547496597, 0.59248606958907868, -0.1101561983478507, 0.49438304650012482, 0.59474678198785869, -1.645394748518235, -0.27115981122198246, -0.21861738242983475, -0.19929484531336172, -0.28078888656203044, -0.22228409421463652, -0.11624367425232722, 0.17537088404295692, 0.40375036773284811, 0.047885615504830094, -0.0022196004727421288, 0.036921631234766591, -0.11152340900803896, 0.12286895345650324, -0.11972728791575733, -0.10179992520841986, 0.13591177376177113, 0.50338173863693902, -1.2835692446168783, -0.17875002246961799, -0.28140066701445438, -0.040946618632095054, -0.074463605922010262, -0.37466113281644425, -0.1147220420576401, 0.052988411479887421, 0.14677410045652142, 0.39011534370267248, 0.15912575315402386, 0.084834199870660251, -0.10277427452708562, -0.085559485088563855, -0.11682415045479322, 0.05262141084810093, 0.20549078034481422, -0.41878000051588476, 1.4050901028226914, -0.27579731620696935, -0.61010467761892828, -0.16701655553809494, -0.1144709441785222, -0.34614240248871647, -0.65981020048738059, 0.13235790865381103, 1.1328389671804187, -1.3597656649748362, -0.3094466900106812, 0.55626440576122715, 0.098019884523595807, 0.064469767708162609, 0.026065506852374003, 0.25357493732084352, 0.070465688161613152, 0.60549463343493781, 0.38894451598242352, 0.055968113742291437, -0.25128214958395512, -0.28268245477327719, -0.44246086640214494, -0.75909401317822733, -0.51367661660588881, 0.49689296226288548, 1.8112939645565271, 0.29980923971621182, -0.13968124898541612, 0.11126711357472518, 0.25559308606692444, -0.32097813347008325, -0.060477745877720118, -0.0090979731356993308, -0.16246175176326438, 1.8670820686389544, -0.05372518930927217, 0.28123701080102348, 0.33576810047488798, 0.092451618159746646, 0.66274486752887019, 0.86383071901713038, -2.7254841648096022, 2.6382882397430767, 10.845421102893114, -0.072641148474023753, 0.022200686756774578, 0.0041946420423329585, -0.08501449620843872, 0.16571470237865454, 0.08123298302161848, -0.10328213032278777, -0.1967604933313985, -0.23296458597690003, -0.02629859553365458, -0.5042823686504857, -0.084639122024869631, 0.10808549193551432, -0.13876506023853566, -0.082849598169813202, -0.04908691358375443, 0.042273953099003676, 0.65165950441438802, -0.3295822623510693, 0.50816010066624717, -0.19269509163560716, -0.035625093794776533, -0.043617193811499734, 0.24903478760626377, -0.29696767903277321, 0.10979569614347554, 0.47903133036155277, 2.2432386607122168, 0.27507031865189274, 0.41056502851111143, 0.056320697486546313, 0.42492974308398074, 0.49116975520311107, -0.0036969701664376622, -0.36179170141914047, 0.10564044142666751, -0.092023333954944039, 0.17530065130722744, -0.10757927374864482, 0.13216674341212517, -0.17105969567801693, 0.021553330311021947, -0.29625155432128397, 0.048926722749586475, -0.044247348735510131, -0.001369944354796605, -0.020909939290641784, 0.07659709423453058, -0.07914717121798448, 0.18546692204124571, 0.046369218103846255, -0.059825065473612889, -0.2270179231869161, 0.13730454683241503, -0.067060655503705643, 0.22100342759824576, 0.14502598059960239, -0.060061328266329803, -0.12884031483778818, -0.15468487909747949, -0.16003806693852579, -0.1762718701746277, -0.077354692377703801, 0.09350812122874165, -0.27427733812389737, -0.20252028602512329, 0.013449218260293671, 0.15862144769954384, 0.25073457725985643, -0.0039840373317751917, -0.046521514696202595, 0.15091696476313418, 21.029515136159894]

In [6]:
def linear_regression_one(X, Y, hr, learning_rate):
    iteration = 5000
    egularized
    #W_b = np.random.rand(feature_num*hr+1)
    W_b = W_b_XX
    y_len = X.shape[1] - hr
    #print(count_loss(W_b, X, Y, hr))
    for epoch in range(iteration):
        if epoch % 10 == 0:
            print('epoch:', (epoch+1), 'RMSD:', count_loss(W_b, X, Y, hr))
            if (epoch+1) % 500 == 0:
                print(list(W_b))
        temp_W_b = W_b
        for i in range(len(W_b)):
            delta = 0
            for cntr in range(y_len):
                if (cntr + hr) % 480 == 0 and cntr is not 0:
                    cntr += 10
                x = np.concatenate((np.array(X[:,cntr:cntr+hr]).reshape(-1), [1]), axis=0)
                delta += (np.dot(W_b, x) - Y[cntr+hr]) * x[i]
            temp_W_b[i] = W_b[i] - learning_rate * (1 / y_len) * delta
        W_b = temp_W_b
        if epoch % 1000 == 0:
            learning_rate *= 0.1
            
    return W_b

def count_loss(W_b, X, Y, hr):
    loss = 0.
    y_len = X.shape[1] - hr
    RMSE = 0.
    for cntr in range(y_len):
        if (cntr + hr) % 480 == 0 and cntr is not 0:   
            cntr += 10
        x = np.concatenate((np.array(X[:,cntr:cntr+hr]).reshape(-1), [1]), axis=0)
        loss += ((np.dot(W_b, x)) - Y[cntr+hr])**2
        
    return loss

def test(test_data, W_b):
    answer = []
    for data in test_data:
        x_matrix = data.test_get_matrix()
        x_ = np.concatenate((np.array(x_matrix).reshape(-1), np.ones(shape=(162))), axis=0)
        answer.append(np.dot(W_b, x_))
    return answer

def output_answer(output_answer_path, answer):
    #os.remove(output_answer_path)
    #os.mknod("answer.csv")
    with open(output_answer_path, 'a') as output_csv:
        output_csv.write('id,value\r\n')
        for i in range(len(answer)):
            line = 'id_%d,%f\r\n' % (i, answer[i])
            output_csv.write(line)

In [7]:
learning_rate = 0.1
hour = 9
X, Y, feature_mean_dict, feature_std_dict = PM2point5_dataset(train_data_path)
X_ = read_test_data(test_data_path, feature_mean_dict, feature_std_dict)
W = linear_regression_one(X, Y, hour, learning_rate)

NameError: name 'egularized' is not defined

In [9]:
W200 = [-3.2403608763405081, 1.4973998896778131, 0.64659497855238279, 0.11093640121871151, 0.29587130557247837, 0.33250536811036518, -0.075872225006881372, -0.0026644749568973617, 0.23103147229925755, -1.3663852286074538, -0.28138085549084529, 0.04612545676278848, 0.014162945162221396, -0.094679880257011753, -0.075791932009684648, 0.037547133338781007, -0.052306528269237741, 0.44299003773090995, 0.096494800293756344, -0.06542516155900642, 0.070114771756723246, -0.12590812968462425, 0.13746052335761694, -0.091037241242376568, -0.12696445900933689, 0.12457321598045366, 0.43734466629149499, -1.2565008380797595, -0.033174496998173104, -0.15462538562637604, 0.15939322762594441, 0.18888038944606186, -0.26492463975383385, 0.12007289937062023, -0.11638048814577952, 0.12662852798045743, 0.78894998550803919, 0.02252455738812479, 0.33471164116559937, -0.027861222056320203, -0.084560673931068253, -0.15889593885670944, 0.179271600258263, 0.21729870035591786, -0.45663594634596105, 2.4031354403729632, -0.44274714994506031, 0.14641559316220645, -0.2159554386462626, -0.19630581344773948, -0.020938076241565906, -0.44907757076318955, -0.19019606968017214, 0.959238079656064, -2.3992275914275751, -0.20064388170307026, -0.20155435134149263, -0.035876907636342666, 0.13676344721934483, -0.051943995675100801, -0.21115363962722306, 0.13870458286128659, 0.8819810666154797, 0.57192618772125769, -0.31689956058734037, 0.00055469741780321958, -0.44715972558894518, -0.083724194962689277, -0.72407233802668924, -0.50810199322579652, 0.12706628015468979, 1.8512262865354983, -0.02410127735937341, 0.12332741465721457, 0.13626256723435204, 0.11474622717367837, -0.44324492409618749, 0.29571899167334287, -0.027030021997444441, -0.40722666123951518, 1.7290855456068037, -0.075705743707212392, 0.31788879710750217, 0.47585682047902977, -0.58821358858059025, 0.7683457982363634, 2.6317070679350283, -5.1470554269154487, 1.8182702740425893, 13.233545549907076, -0.033868232381851666, 0.012571756492550729, 0.029925892410524606, -0.15597176148481595, 0.14911747945010514, 0.11986078426390417, -0.090714457323535988, -0.17056372007028869, -0.2395398648517876, -0.3365256512852024, 0.10020802794450408, -0.12913736435228249, 0.088191202421828957, 0.078707642693237781, -0.09328743211323938, -0.35724513460594337, 0.11603990123758447, 0.33755702338306232, -0.42698515036443402, 0.54165033440396448, -0.19896641041542246, -0.047852284811599234, 0.010456433015647811, 0.15467017232371874, -0.15069791157847914, 0.0439737399159199, 0.40131896589709476, 1.9103456492273181, 0.25251199517646999, 0.090352477255517746, -0.2826160924661737, 0.086334164447476963, 0.31388875943413708, -0.15273166054826134, -0.091812360430602993, -0.014311942667542576, -0.0059964445356245994, 0.24955199165714306, -0.067043585420157883, 0.21661326344564211, -0.04143731227192031, 0.10804204313344927, -0.2392402845403028, 0.12640399519905812, 0.011723012156906598, -0.13266178455199257, -0.095174794462239157, 0.014243728437689859, -0.20715681714779172, 0.10591334607949239, 0.037622575844815702, -0.045727124703778363, -0.26492218912439675, 0.095531733793805071, -0.11885462235511618, 0.019438342976439864, 0.1952238335290446, 0.018931099090506647, -0.12437531267622712, -0.1689704725821603, -0.11480140084374922, -0.058998298374870674, -0.042229417767810716, -0.0093843912736552952, 0.071099032017382713, -0.17311917975816751, -0.15846016791595621, 0.10034585202468076, 0.31275800914657281, -0.010207969393820095, -0.16447237349090285, 0.087325273974347778, -62.962182034436744, 0.035770913563889485, 0.5574900433044323, -0.0014632040785751072, 0.22673123617153484, 0.66318528372317209, 0.19243810368887984, 0.31300277946326144, 0.49942560975913014, 0.61216085482730764, 0.89203467185273944, 0.56675256269977647, 0.038159215175771616, 0.26878697230578463, 0.12953707774132517, 0.82046866562897969, 0.058393594245834471, 0.64246420092684464, 0.94306057309991986, 0.59285830414095153, 0.8597701762849963, 0.11025104645382532, 0.29593491285993623, 0.69722791871460665, 0.32716549816997609, 0.6308845615811437, 0.023526847938037269, 0.43184165641336886, 0.44500538935271206, 0.078835440888183506, 0.42564355679430954, 0.38799760155015278, 0.96763655310477492, 0.86397765230237422, 0.94703708160879851, 0.8964797850878341, 0.4644432693408333, 0.1568232873173758, 0.55156025725420188, 0.59710914671142146, 0.97985052839129716, 0.36764477388542716, 0.7732656994501782, 0.87924144418008154, 0.90269151727179509, 0.49888344614453484, 0.57890498942812529, 0.022314672580141671, 0.15391668687443702, 0.75388976387022999, 0.98024210931351774, 0.56657423634132154, 0.5433177805310041, 0.42425431593068008, 0.34073858345545749, 0.33802869755617593, 0.94644855828044827, 0.20526786046460399, 0.38399487033851698, 0.40644193355563468, 0.37486802754812931, 0.32725878316026019, 0.72682091762607037, 0.33115047274524551, 0.2532316568393253, 0.23719522222337844, 0.87496368266692748, 0.24401806185674599, 0.1042848902261497, 0.7346351690908437, 0.37106873509638955, 0.92148152730521204, 0.96318353705600102, 0.27762045222737147, 0.88811006407149529, 0.86191752597157367, 0.82806938349027803, 0.35631154244046143, 0.52595392111783723, 0.88483930104614528, 0.58077446734658722, 0.68910495760791413, 0.88027738829491298, 0.78241439971971227, 0.16487726751521165, 0.70460140035641683, 0.059590997284994222, 0.040840166300053431, 0.5020083655090789, 0.44521199966685177, 0.92429600945739665, 0.85813710046667402, 0.42816363790262535, 0.23608593711564133, 0.57337900721831658, 0.30937683930940929, 0.53541286642843589, 0.23448366101524523, 0.4464005879373637, 0.79457968692086323, 0.94494788586000089, 0.48947837171378228, 0.036629095852609692, 0.41268033354836753, 0.86700806798052887, 0.058942170919201245, 0.48235935285726828, 0.66659420300024341, 0.54542503360930739, 0.97406650733290689, 0.6520352797718999, 0.50173738913786525, 0.39555377339294501, 0.1092107724114763, 0.43989304744781837, 0.72419634589926407, 0.70877951459332389, 0.46378262856066971, 0.80640384935762377, 0.54109490490117451, 0.4506999351788834, 0.3131996012358243, 0.3097168711403861, 0.93647379854708279, 0.96371900150131973, 0.46354416127453774, 0.91317671127071176, 0.90130642517175719, 0.98417747456358284, 0.8750856945292097, 0.64155851473935643, 0.73110076606220653, 0.84893359707576455, 0.25034071457698026, 0.41207545729374301, 0.057649734698234381, 0.50110471431829517, 0.85542447107144193, 0.34321936349284971, 0.45281032229795315, 0.67594656790809393, 0.42014710562155361, 0.61245311019425974, 0.87282876062686798, 0.30284891785252405, 0.72033535387346947, 0.23394763173292679, 0.2770328390057254, 0.44301500486533274, 0.49900225832253403, 0.38171221262424931, 0.88145772556695334, 0.62430859340788558, 0.54189281039396719, 0.065137782960085086, 0.41743136913316031, 0.87417446128626386, 0.1796101554164693, 0.54175343712840507, 0.16536589547059685, 0.97952217830081578, 0.089173312939628629]
answer = test(X_, W200)
output_answer(output_answer_path, answer)

In [8]:
W_b_200 = [-2.6624858238458513, 1.0309398024423948, 0.62637502483535623, 0.053873889965485755, 0.41546304207422563, -0.23032474454570828, -0.068225019801408771, -0.026532337789027298, 0.67365289500290337, -1.349034564519995, -0.16226475882785829, 0.039976733216226058, -0.18888176848699584, -0.20141393464866747, -0.10011309784926883, -0.02148337478998958, 0.10413391684210457, 0.50193850678344198, 0.085241321986684895, -0.062432173981869538, 0.071973298377670566, -0.12616176659760853, 0.15176992394926489, -0.092634443917966686, -0.12225503664151782, 0.12012326408614349, 0.43410070401101952, -1.2242527291466656, 0.065073811903353401, -0.12915334429010611, -0.044272033340217051, 0.055488441387344309, -0.23879841102285085, 0.044928730108652865, 0.021270116337479661, 0.18467978346917238, 0.7228320883397279, -0.038832520756147591, 0.07199254317277351, -0.098232759799199237, -0.19364663697911, -0.11080297256624543, 0.11258229272487315, 0.30220728016303489, -0.3953224564537266, 2.3200008322567136, -0.75035336456834711, -0.63759015694167775, -0.3395345567439817, -0.24678734018461471, -0.11967715414335602, -0.61893692717384496, 0.10450176102621819, 1.0973646643267008, -2.2528775127962151, 0.11431991344053759, 0.6290302190467727, 0.22607827542003125, 0.36672073502019775, -0.067234901378686165, 0.01210669662288682, -0.21370131180861435, 0.70492043468852428, 0.36243834423255528, -0.075832647088857758, -0.15086750100242299, -0.27536409662021727, -0.16811829503801498, -0.64078029010124915, -0.62731257752057545, 0.132312629362558, 1.9072063650029796, -0.10249923913906255, 0.24476725503195668, 0.079928066347798338, 0.026488325264998589, -0.28591690626790461, 0.22470655535048259, -0.07099053401008229, -0.31514445347815057, 1.688197316872857, -0.044021171005123032, 0.37635005804696331, 0.32862890980576831, -0.54585489565085765, 0.89849399619402481, 2.4594555161398506, -5.1516423499121453, 2.0129277200509419, 13.096257481866544, -0.024930767025303127, -0.0098073965743275919, 0.019313282954949496, -0.14201010036143932, 0.16175920107279773, 0.12196983236587845, -0.092211875350010963, -0.18419509831500397, -0.24024603908842926, -0.51089289436707108, 0.25871892507920446, 0.11859887384109846, -0.10793553139628681, -0.0063486694617393485, -0.21840973425656973, -0.48692052697737631, 0.27580335087752639, 0.47362629069901452, -0.46595109189199363, 0.58976785872790383, -0.18637032970632031, -0.053794339364275502, -0.047851742721882258, 0.20926214255753658, -0.18409908884503634, 0.048803857323550533, 0.41770319735846506, 1.8621801606528934, 0.11055782578354302, 0.1105625498902427, 0.017119715987176834, 0.21203533967887192, 0.35832046800329614, -0.071029242022784239, -0.32126138110365532, -0.087057114905128671, -0.021723735146029547, 0.24748027369327102, -0.060868534058875864, 0.22822244065637279, -0.041888373954462522, 0.10402888604577802, -0.24055647910485467, 0.12078492118050847, 0.0038096996034921666, -0.13530375377012566, -0.083869619338199419, 0.019739656794677447, -0.21032838739994472, 0.11800392623401773, 0.047488475137958404, -0.043906494140242149, -0.26833098506689701, 0.087573632140131349, -0.14211066075469622, 0.043287560372598261, 0.21348916098305179, 0.054694062717239547, -0.15582748917930658, -0.17564665554915904, -0.10344640387794778, -0.055840390851103233, -0.027256098511305692, 0.030611773786059358, 0.026307038634456305, -0.18245790604016704, -0.20020165222904185, 0.11970409628056237, 0.33824905508459574, -0.022674045482961761, -0.14479021944003562, 0.053235624512595461, -58.312229625056823, 0.76173076451297328, 0.58780563370997629, 0.1265637823348586, 0.71498129805198307, -0.084923234967709677, 0.60585595111040091, 0.098595956247449698, 0.51351242569694611, 0.48676949547879417, 0.011367981745760054, 0.7494585172733681, 0.88502711814028201, 0.51714880569571897, 0.85411111536209827, 0.93836703555163403, 0.80720162086887448, 0.74102985452835679, 0.32335408043906266, 0.85426596021015866, 0.10088558488598967, 0.5784066494278467, 0.077441848137244126, 0.35933159943472737, 0.67867643899914154, 0.45264211292071749, 0.13350270503671219, 0.88146509436964959, 0.016340677478727673, 0.052353659612596012, 0.15972510061962583, 0.60183936868016596, 0.84932767015909949, 0.41067324349768702, 0.62936135825593986, 0.69913613495120364, 0.91220165570693057, 0.73977551184301316, 0.31179376139317549, 0.25504845949332616, 0.86759192700422338, 0.57216027318836549, 0.22018090734000673, 0.25892213028345001, 0.66649298125589196, 0.89603304228116132, 0.17338580295706979, 0.11248603576442912, 0.51116693214907027, 0.34491743991122137, 0.59007731466443758, 0.065265053103742296, 0.5596454676641831, 0.52660176511657053, 0.43765809447930371, 0.62807903990214531, 0.5761731441853265, 0.1428268715600515, 0.927014907057372, 0.60132362386664151, 0.3642723403559236, 0.08559724071233614, 0.65115551580496922, 0.15655695056725796, 0.67256762345583476, 0.42095292994407602, 0.95416379497114234, 0.50625488061749924, 0.5956343602804286, 0.27300633449204298, 0.45842693538238399, 0.10340828515398082, 0.53750852065725452, 0.73884358419412721, 0.31272074724892868, 0.08281659884503885, 0.30642013663941015, 0.33036913381995836, 0.84443228505540624, 0.8273659229938517, 0.18085980848139843, 0.5827050647093488, 0.045461386185662103, 0.13611882946085377, 0.81036759195686159, 0.88513145052803088, 0.93326447463667395, 0.56004836870671648, 0.35893596811306144, 0.65026764270817294, 0.64738631778414257, 0.45505290408855331, 0.61758374868580546, 0.43092014280599344, 0.51234225167814418, 0.21828002374572186, 0.45611974222730378, 0.69105247657004032, 0.92415236306405479, 0.53731307393756844, 0.76217384952734357, 0.79639465241974339, 0.22630898454348497, 0.80674469597661391, 0.51559951076082478, 0.83418126125563907, 0.73282562580853439, 0.21358506308277636, 0.46276056839630125, 0.67206729501067852, 0.026297693171989107, 0.12217524540814816, 0.88042523323866395, 0.88152942913188426, 0.24326375103918946, 0.90626176398314406, 0.4259319613272991, 0.51680972467454844, 0.59009639169692718, 0.23448303165013945, 0.057755830886266145, 0.22402203682835994, 0.70430770104245433, 0.37831721615095498, 0.33745739595991575, 0.1604228625946644, 0.33761907477745351, 0.62310571367012546, 0.4024806010674033, 0.10209972532907902, 0.70392683133218148, 0.64149625952063571, 0.33042382219139765, 0.13182226177187048, 0.21437534072200576, 0.80114365169271562, 0.52541331183172901, 0.36921149648180979, 0.61295881743894975, 0.17120967021263486, 0.046212582426573452, 0.80489043294131224, 0.1704206395773413, 0.83651750304088979, 0.34676538915572508, 0.61923883295945159, 0.63853500787739925, 0.18547424056352513, 0.60796326254458122, 0.030294369333161104, 0.78973258744959918, 0.85461965602396761, 0.27003796485013509, 0.96625886844308195, 0.5193182069964144, 0.93095049893294923, 0.70015321942989683, 0.40573981438488455, 0.50076246402026825, 0.84581853833335652, 0.43651375869997011, 0.59655121858320803]
print(len(W_b_200))
answer = test(X_, W_b_200)
output_answer(output_answer_path, answer)

324
